##### Import libraries 

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
base_url='https://axar.az/'

In [3]:
def get_soup(url):
    
    while True:
        
        try:
            request=requests.get(url)
            
            content=request.content
            
            soup=BeautifulSoup(content, 'html.parser')
            
            return soup
            
        except:
            
            pass

##### Categories

In [4]:
def get_categories(url):
    
    soup=get_soup(url)
    
    div=soup.find('div', {"class":"menu1"})
    
    li_s=div.find_all('li')
    
    categories={row.text: base_url+row.find('a').get('href') for row in li_s}
    
    return categories

In [5]:
categories_name_and_urls=get_categories('https://axar.az/')

print(categories_name_and_urls)

{'Gündəm': 'https://axar.az//news/gundem/', 'Siyasət': 'https://axar.az//news/siyaset/', 'Planet': 'https://axar.az//news/planet/', 'Toplum': 'https://axar.az//news/toplum/', 'İqtisadiyyat': 'https://axar.az//news/iqtisadiyyat/', 'Kult': 'https://axar.az//news/kult/', 'Sağlıq': 'https://axar.az//news/saghliq/', 'Reportaj': 'https://axar.az//news/layiheler/', 'Yazarlar': 'https://axar.az//news/yazarlar/', 'Maqazin': 'https://axar.az//news/maqazin/', 'İdman': 'https://axar.az//news/idman/', 'Avto': 'https://axar.az//news/avto/', 'Foto': 'https://axar.az//news/foto/', 'Video': 'https://axar.az//news/video/'}


In [6]:
def get_alt_categories(url):
    
    soup=get_soup(url)
    
    alt_cat=soup.find('ul', {'class':'menu1 no_background'})
    
    if not alt_cat:
        return None
    
    alt_cats={row.text: base_url+row.get('href') for row in alt_cat.find_all('a') if row.text!="Ana səhifə"}
    
    return alt_cats

In [7]:
for category_name in list(categories_name_and_urls.keys()):
    
    category_url=categories_name_and_urls[category_name]
    
    alt_cat=get_alt_categories(category_url)
    
    if not alt_cat:
        pass
    
    else:
        categories_name_and_urls.update(alt_cat)

In [8]:
print(categories_name_and_urls)

{'Gündəm': 'https://axar.az//news/gundem/', 'Siyasət': 'https://axar.az//news/siyaset/', 'Planet': 'https://axar.az//news/planet/', 'Toplum': 'https://axar.az//news/toplum/', 'İqtisadiyyat': 'https://axar.az//news/iqtisadiyyat/', 'Kult': 'https://axar.az//news/kult/', 'Sağlıq': 'https://axar.az//news/saghliq/', 'Reportaj': 'https://axar.az//news/layiheler/', 'Yazarlar': 'https://axar.az//news/yazarlar/', 'Maqazin': 'https://axar.az//news/maqazin/', 'İdman': 'https://axar.az//news/idman/', 'Avto': 'https://axar.az//news/avto/', 'Foto': 'https://axar.az//news/foto/', 'Video': 'https://axar.az//news/video/', 'Ədəbiyyat': 'https://axar.az//news/edebiyyat/', 'Musiqi': 'https://axar.az//news/musiqi/', 'Sinema': 'https://axar.az//news/sinema/', 'Rəsm': 'https://axar.az//news/resm/', 'Teatr': 'https://axar.az//news/teatr/', 'Müsahibə': 'https://axar.az//news/musahibe/', 'Hadisə': 'https://axar.az//news/hadise/', 'Anım': 'https://axar.az//news/anim/'}


In [9]:
categories_name_and_urls.pop('Gündəm')

categories_name_and_urls.pop('Foto')

categories_name_and_urls.pop('Video')

print(categories_name_and_urls)

{'Siyasət': 'https://axar.az//news/siyaset/', 'Planet': 'https://axar.az//news/planet/', 'Toplum': 'https://axar.az//news/toplum/', 'İqtisadiyyat': 'https://axar.az//news/iqtisadiyyat/', 'Kult': 'https://axar.az//news/kult/', 'Sağlıq': 'https://axar.az//news/saghliq/', 'Reportaj': 'https://axar.az//news/layiheler/', 'Yazarlar': 'https://axar.az//news/yazarlar/', 'Maqazin': 'https://axar.az//news/maqazin/', 'İdman': 'https://axar.az//news/idman/', 'Avto': 'https://axar.az//news/avto/', 'Ədəbiyyat': 'https://axar.az//news/edebiyyat/', 'Musiqi': 'https://axar.az//news/musiqi/', 'Sinema': 'https://axar.az//news/sinema/', 'Rəsm': 'https://axar.az//news/resm/', 'Teatr': 'https://axar.az//news/teatr/', 'Müsahibə': 'https://axar.az//news/musahibe/', 'Hadisə': 'https://axar.az//news/hadise/', 'Anım': 'https://axar.az//news/anim/'}


##### Pages

In [10]:
def get_pages(url):
    
    urls=[]
    
    number=2
    
    while True:
        
        new_url=url+f'page{number}/'
        
        soup=get_soup(new_url)
    
        div=soup.find('div', {'class': 'newsPlace'})
        
        div_body=div.text.strip('\n')
        
        if div_body=='':
            
            break
            
        urls.append(new_url)
        
        number+=1
        
    urls.insert(0, url)
        
    return urls

##### News urls

In [11]:
def get_news_urls(url):
    
    urls=[]
    
    pages=get_pages(url)
    
    for url in pages:
           
        soup=get_soup(url)

        div=soup.find('div', {"class": 'newsPlace'})

        tables=div.find_all('table', {'id': 'catNews'})

        for row in tables:

            url=row.find('div', {"id": "cat_news_title"}).find('a').get('href')

            urls.append(url)
    
    return urls

##### News data

In [12]:
def get_news_data(url):
    
    summary_text, article_text=None, None
    
    soup=get_soup(url)
    
    summary=soup.find('strong')
    
    if summary:
        
        summary_text=summary.text
    
    article=soup.find('span', {'id': 'font_size', 'class': 'article_body'})
    
    if article:
        
        article_text=article.text.strip('\n').strip(summary_text)
    
    return summary_text, article_text

##### Main

In [13]:
result={'category': [], 'summary':[], 'article':[]}

errors=[]

In [14]:
for category_name, category_url in categories_name_and_urls.items():
    
    print(f'{category_name} category started')
    
    news_urls=get_news_urls(category_url)
    
    for url in news_urls:
        
        try:
        
            summary_text, article_text=get_news_data(url)

            result['category'].append(category_name)
            
            result['summary'].append(summary_text)
            
            result['article'].append(article_text)
            
        except:
            
            errors.append(url)
            
    print(f'{category_name} category ended')

Siyasət category started
Siyasət category ended
Planet category started
Planet category ended
Toplum category started
Toplum category ended
İqtisadiyyat category started
İqtisadiyyat category ended
Kult category started
Kult category ended
Sağlıq category started
Sağlıq category ended
Reportaj category started
Reportaj category ended
Yazarlar category started
Yazarlar category ended
Maqazin category started
Maqazin category ended
İdman category started
İdman category ended
Avto category started
Avto category ended
Ədəbiyyat category started
Ədəbiyyat category ended
Musiqi category started
Musiqi category ended
Sinema category started
Sinema category ended
Rəsm category started
Rəsm category ended
Teatr category started
Teatr category ended
Müsahibə category started
Müsahibə category ended
Hadisə category started
Hadisə category ended
Anım category started
Anım category ended


In [15]:
df=pd.DataFrame(result)

In [16]:
df.head()

,category,summary,article
0,Siyasət,Azərbaycan Respublikasının Prezidenti İlham Əl...,"\nAxar.az xəbər verir ki, bu barədə Prezidentin M"
1,Siyasət,Azərbaycan Respublikasının Prezidenti İlham Əl...,17:50\nAzərbaycan Respublikasının Prezidenti İ...
2,Siyasət,"Xəbər verdiyimiz kimi, Azərbaycan Prezidenti İ...","14:22\nXəbər verdiyimiz kimi, Azərbaycan Prezi..."
3,Siyasət,Son üç il ərzində keçirilmiş hərbi əməliyyatla...,"\nAxar.az xəbər verir ki, bu barədə Azərbaycan..."
4,Siyasət,Qırx dörd günlük müharibə zamanı çoxlu xarici ...,"\nAxar.az xəbər verir ki, bu barədə Azərbaycan..."


In [17]:
df.shape

(385527, 3)

In [18]:
df.to_csv('azerbaijani_news_dataset.csv', index=False)